## Plot markers umap embeddings
Does this over each donor and separate

In [1]:
# Input info
allSamples_f <- "/data/Mito_Trace/output/pipeline/DUPI_april08_2021/MTblacklist_mtasnucl_Bonly/data/annotation/mergedSamples/allSamples.rds"
#se_f = "/data/Mito_Trace/output/pipeline/DUPI_april08_2021/MTblacklist_mtasnucl_Bonly/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_init/knn/kparam_3/concat/annotation_clones/SE.rds"
sample_names = "preB,postB"
outdir = "/data/Mito_Trace/output/pipeline/DUPI_april08_2021/MTblacklist_mtasnucl_Bonly/data/annotation/mergedSamples/sepConds"
#"/data/Mito_Trace/output/pipeline/DUPI_april08_2021/MTblacklist_mtasnucl_Bonly/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_init/knn/kparam_3/concat/annotation_clones/umap_markers"

# Parameters
assay="RNA"

# Parallel processing
cores = 4

# R utils script
utils_script <- "/data/Mito_Trace/R_scripts/seurat_utils.R"



In [2]:
source(utils_script)
dir.create(outdir, showWarnings = FALSE)


Attaching SeuratObject



In [3]:
sample_names <- unlist(strsplit(sample_names, ",")[[1]])
sample_names

[1] "preB"  "postB"

In [4]:
allSamples <- readRDS(allSamples_f)
allSamples

[[1]]
An object of class Seurat 
40000 features across 1351 samples within 1 assay 
Active assay: ATAC (40000 features, 0 variable features)

[[2]]
An object of class Seurat 
41874 features across 2890 samples within 1 assay 
Active assay: ATAC (41874 features, 0 variable features)


In [5]:
library(GenomicRanges)
library(Seurat)
library(Signac)
library(GenomeInfoDb)
library(EnsDb.Hsapiens.v75)
library(ggplot2)
library(patchwork)
set.seed(1234)
library(data.table)
library(magrittr)
library(cowplot)
library(metap)
library("RColorBrewer")

library(future)
plan()

plan("multiprocess", workers = cores)
#options(future.globals.maxSize = 50000 * 1024^2) # for 50 Gb RAM
options(future.globals.maxSize = 8000 * 1024^2)

library(dplyr)

Loading required package: stats4

Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tapply,
    union, unique, unsplit, which.max, which.min


Loading required package: S4Vectors


Attaching package: ‘S4Vectors’


The following objects are masked from ‘package:base’:

    expand.grid, I, unname


Loading required package: IRanges

Loading required package: GenomeInfoDb

Loading required package: ensembldb

Loading required package: GenomicFeatures

Loading required package: AnnotationDbi

Lo

sequential:
- args: function (..., envir = parent.frame())
- tweaked: FALSE
- call: NULL

Warning message:
"Strategy 'multiprocess' is deprecated in future (>= 1.20.0). Instead, explicitly specify either 'multisession' or 'multicore'. In the current R session, 'multiprocess' equals 'multicore'."

Attaching package: 'dplyr'


The following objects are masked from 'package:data.table':

    between, first, last


The following objects are masked from 'package:ensembldb':

    filter, select


The following object is masked from 'package:AnnotationDbi':

    select


The following object is masked from 'package:Biobase':

    combine


The following objects are masked from 'package:GenomicRanges':

    intersect, setdiff, union


The following object is masked from 'package:GenomeInfoDb':

    intersect


The following objects are masked from 'package:IRanges':

    collapse, desc, intersect, setdiff, slice, union


The following objects are masked from 'package:S4Vectors':

    first, intersect, rename, setdiff, setequal, union


The following objects are masked from 'package

## Plot markers for separate samples

In [6]:
markers_f <- "/data/Mito_Trace/data/processed/pbmc_markers.txt"
markers <- read.csv(markers_f, header=F)[[1]]

In [8]:
small.umap.outdir <- file.path(outdir, "markersDimRed")
dir.create(small.umap.outdir)


Warning message in dir.create(small.umap.outdir):
"'/data/Mito_Trace/output/pipeline/DUPI_april08_2021/MTblacklist_mtasnucl_Bonly/data/annotation/mergedSamples/sepConds/markersDimRed' already exists"


In [7]:
featplot <- function(name.sig, se, curr.outdir, feat.names=NULL){
    if (!is.null(feat.names)){
        name <- feat.names[name.sig,]
    }else{
        name <- name.sig
    }
    if (name.sig %in% rownames(se)){
    feat <- FeaturePlot(se,  features=name.sig) + ggtitle(name)
    ggsave(plot=feat,
           file=file.path(curr.outdir, paste0(name.sig,".embedFeat.top.png")))
        }
    }

In [9]:
run.sample.sep <- function(curr.sample){
    curr.outdir <- file.path(small.umap.outdir, curr.sample$orig.ident[[1]])
    dir.create(curr.outdir)
    out <- embed.atac(curr.sample)
    curr.sample <- out[[1]]
    DimPlot(curr.sample)
    ggsave(file.path(curr.outdir, "dimPlot.png"))
    feature.names <- ClosestFeature(curr.sample, rownames(curr.sample))
    marker.feature.names <- feature.names %>% filter(gene_name %in% markers)
    rownames(marker.feature.names) <- marker.feature.names[,"query_region"]
    # marker.feature.names

    marker.feature.names$name <- paste0(marker.feature.names$gene_name, "_", marker.feature.names$type)

    sapply(rownames(marker.feature.names), 
            FUN=featplot, se=curr.sample, curr.outdir=curr.outdir, feat.names=marker.feature.names["name"])
}

In [12]:
sapply(allSamples, FUN=run.sample.sep)

Performing TF-IDF normalization

Running SVD

Scaling cell embeddings

Warning message:
"The default method for RunUMAP has changed from calling Python UMAP via reticulate to the R-native UWOT using the cosine metric
To use Python UMAP via reticulate, set umap.method to 'umap-learn' and metric to 'correlation'
This message will be shown once per session"
20:03:20 UMAP embedding parameters a = 0.9922 b = 1.112

20:03:20 Read 1351 rows and found 49 numeric columns

20:03:20 Using Annoy for neighbor search, n_neighbors = 30

20:03:20 Building Annoy index with metric = cosine, n_trees = 50

0%   10   20   30   40   50   60   70   80   90   100%

[----|----|----|----|----|----|----|----|----|----|

*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
|

20:03:20 Writing NN index file to temp file /tmp/Rtmp6rPzRB/file1e98a34eceecd2

20:03:20 Searching Annoy index using 4 threads, search_k = 3000

20:03:20 Annoy recall = 100%

20:03:21 Commencing

chr1-161523797-161524588,/data/Mito_Trace/output/pipeline/DUPI_april08_2021/MTblacklist_mtasnucl_Bonly/data/annotation/mergedSamples/sepConds/markersDimRed/preB/chr1-161523797-161524588.embedFeat.top.png,/data/Mito_Trace/output/pipeline/DUPI_april08_2021/MTblacklist_mtasnucl_Bonly/data/annotation/mergedSamples/sepConds/markersDimRed/postB/chr1-161523797-161524588.embedFeat.top.png
chr1-161529790-161531534,/data/Mito_Trace/output/pipeline/DUPI_april08_2021/MTblacklist_mtasnucl_Bonly/data/annotation/mergedSamples/sepConds/markersDimRed/preB/chr1-161529790-161531534.embedFeat.top.png,/data/Mito_Trace/output/pipeline/DUPI_april08_2021/MTblacklist_mtasnucl_Bonly/data/annotation/mergedSamples/sepConds/markersDimRed/postB/chr1-161529790-161531534.embedFeat.top.png
chr1-161539721-161540821,/data/Mito_Trace/output/pipeline/DUPI_april08_2021/MTblacklist_mtasnucl_Bonly/data/annotation/mergedSamples/sepConds/markersDimRed/preB/chr1-161539721-161540821.embedFeat.top.png,/data/Mito_Trace/output/pipeline/DUPI_april08_2021/MTblacklist_mtasnucl_Bonly/data/annotation/mergedSamples/sepConds/markersDimRed/postB/chr1-161539721-161540821.embedFeat.top.png
chr1-161541921-161542393,/data/Mito_Trace/output/pipeline/DUPI_april08_2021/MTblacklist_mtasnucl_Bonly/data/annotation/mergedSamples/sepConds/markersDimRed/preB/chr1-161541921-161542393.embedFeat.top.png,/data/Mito_Trace/output/pipeline/DUPI_april08_2021/MTblacklist_mtasnucl_Bonly/data/annotation/mergedSamples/sepConds/markersDimRed/postB/chr1-161541921-161542393.embedFeat.top.png
chr1-161555025-161555205,/data/Mito_Trace/output/pipeline/DUPI_april08_2021/MTblacklist_mtasnucl_Bonly/data/annotation/mergedSamples/sepConds/markersDimRed/preB/chr1-161555025-161555205.embedFeat.top.png,/data/Mito_Trace/output/pipeline/DUPI_april08_2021/MTblacklist_mtasnucl_Bonly/data/annotation/mergedSamples/sepConds/markersDimRed/postB/chr1-161555025-161555205.embedFeat.top.png
chr1-161561347-161561621,/data/Mito_Trace/output/pipeline/DUPI_april08_2021/MTblacklist_mtasnucl_Bonly/data/annotation/mergedSamples/sepConds/markersDimRed/preB/chr1-161561347-161561621.embedFeat.top.png,/data/Mito_Trace/output/pipeline/DUPI_april08_2021/MTblacklist_mtasnucl_Bonly/data/annotation/mergedSamples/sepConds/markersDimRed/postB/chr1-161561347-161561621.embedFeat.top.png
chr1-161566382-161566527,/data/Mito_Trace/output/pipeline/DUPI_april08_2021/MTblacklist_mtasnucl_Bonly/data/annotation/mergedSamples/sepConds/markersDimRed/preB/chr1-161566382-161566527.embedFeat.top.png,/data/Mito_Trace/output/pipeline/DUPI_april08_2021/MTblacklist_mtasnucl_Bonly/data/annotation/mergedSamples/sepConds/markersDimRed/postB/chr1-161566382-161566527.embedFeat.top.png
chr5-35852400-35852876,/data/Mito_Trace/output/pipeline/DUPI_april08_2021/MTblacklist_mtasnucl_Bonly/data/annotation/mergedSamples/sepConds/markersDimRed/preB/chr5-35852400-35852876.embedFeat.top.png,/data/Mito_Trace/output/pipeline/DUPI_april08_2021/MTblacklist_mtasnucl_Bonly/data/annotation/mergedSamples/sepConds/markersDimRed/postB/chr5-35852400-35852876.embedFeat.top.png
chr5-35853491-35855028,/data/Mito_Trace/output/pipeline/DUPI_april08_2021/MTblacklist_mtasnucl_Bonly/data/annotation/mergedSamples/sepConds/markersDimRed/preB/chr5-35853491-35855028.embedFeat.top.png,/data/Mito_Trace/output/pipeline/DUPI_april08_2021/MTblacklist_mtasnucl_Bonly/data/annotation/mergedSamples/sepConds/markersDimRed/postB/chr5-35853491-35855028.embedFeat.top.png
chr5-35856179-35857318,/data/Mito_Trace/output/pipeline/DUPI_april08_2021/MTblacklist_mtasnucl_Bonly/data/annotation/mergedSamples/sepConds/markersDimRed/preB/chr5-35856179-35857318.embedFeat.top.png,/data/Mito_Trace/output/pipeline/DUPI_april08_2021/MTblacklist_mtasnucl_Bonly/data/annotation/mergedSamples/sepConds/markersDimRed/postB/chr5-35856179-35857318.embedFeat.top.png
chr5-35857832-35858112,/data/Mito_Trace/output/pipeline/DUPI_april08_2021/MTblacklist_mtasnucl_Bonly/data/annotation/mergedSamples/sepConds/markersDimRed/preB/chr5-35857832-35858

In [13]:
sessionInfo()

R version 4.1.1 (2021-08-10)
Platform: x86_64-conda-linux-gnu (64-bit)
Running under: Ubuntu 20.04.3 LTS

Matrix products: default
BLAS/LAPACK: /home/isaac/miniconda3/envs/mttrace/lib/libopenblasp-r0.3.18.so

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

attached base packages:
[1] stats4    stats     graphics  grDevices utils     datasets  methods  
[8] base     

other attached packages:
 [1] dplyr_1.0.7               future_1.23.0            
 [3] RColorBrewer_1.1-2        metap_1.1                
 [5] cowplot_1.1.1             magrittr_2.0.1           
 [7] data.table_1.14.2         patchwork_1.1.1          
 [9] ggplot2_3.3.5             EnsDb.Hsapiens.v75_2.99.0
[11] 